In [3]:
import tensorflow as tf
import tensorflow_probability as tfp
import pandas as pd
import random
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from IPython.display import clear_output
from scipy.interpolate import interp1d
from datetime import datetime
import os

import numpy as np
from tensorflow import keras
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
tfd = tfp.distributions

import keras
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten

# from photoz_utils import *
# from photoz_plots import *

Num GPUs Available:  1


In [4]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=10000)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [14]:
z_max = 2.5
sample_array = "[1, 1, 1, 1]"
sampling_method = 'control'

if sampling_method == 'control':
    train_array = pd.read_csv('/data/HSC/HSC_v6/3_model_comparison/'+str(sampling_method)+'_training_set_'+str(z_max)+'z#1.csv')
else:
    train_array = pd.read_csv('/data/HSC/HSC_v6/3_model_comparison/'+str(sampling_method)+'_training_set_'+str(z_max)+'z_'+str(sample_array)+'#1.csv')

train_array = np.asarray(train_array)

test_array = pd.read_csv('/data/HSC/HSC_v6/3_model_comparison/testing_'+str(z_max)+'z_.csv')
test_array = np.asarray(test_array)

In [15]:
# train_array['specz_redshift'].hist(bins=4)

In [16]:
OID_test = test_array[:,0]
X_test = list(test_array[:,[12,13,14,15,16]])
y_test = list(test_array[:,2])

In [17]:
X_train = list(train_array[:,[12,13,14,15,16]])
y_train = list(train_array[:,2])
scaler = preprocessing.StandardScaler()
scaler.fit_transform(X_train)

array([[ 0.03320265, -0.03945451, -0.14544286, -0.22768356, -0.30414548],
       [-0.77810049, -0.76156452, -0.74188111, -0.7624553 , -0.74727601],
       [-1.20175264, -0.93298335, -0.78624739, -0.67422576, -0.62954758],
       ...,
       [-1.65984707, -1.50395633, -1.43617716, -1.39843736, -1.37911015],
       [-0.12843767, -0.41030769, -0.46107987, -0.49235366, -0.5102752 ],
       [ 0.38328757,  0.03217742, -0.12393714, -0.15955881, -0.15472609]])

In [18]:
X_test = np.asarray(X_test).astype(np.float32) # need to add this so tensorflow doesn't complain
y_test = np.asarray(y_test).astype(np.float32)
scaler.transform(X_test)

array([[ 1.522279  ,  1.9762381 ,  2.424137  ,  2.4398282 ,  2.5106144 ],
       [-1.1364897 , -1.266366  , -1.2827677 , -1.291099  , -1.2837365 ],
       [ 0.21055032, -0.13369519, -0.26357216, -0.28498447, -0.2850285 ],
       ...,
       [-0.4785226 ,  0.03405207,  0.2040236 ,  0.31363428,  0.39234737],
       [ 1.6291894 ,  1.9738779 ,  2.1211395 ,  1.9642758 ,  1.8407187 ],
       [ 0.57786137,  0.8812015 ,  0.8158645 ,  0.9164957 ,  0.97092205]],
      dtype=float32)

In [19]:
X_train = np.asarray(X_train).astype(np.float32) # need to add this so tensorflow doesn't complain
y_train = np.asarray(y_train).astype(np.float32)

In [20]:
# From Evan's code, didn't completely understand this part
def posterior_mean_field(kernel_size: int, bias_size: int, dtype: any) -> tf.keras.Model:
    """Specify the surrogate posterior over `keras.layers.Dense` `kernel` and `bias`."""
    n = kernel_size + bias_size
    c = np.log(np.expm1(1.))
    return tf.keras.Sequential([
        tfp.layers.VariableLayer(2 * n, dtype=dtype, initializer=lambda shape, dtype: random_gaussian_initializer(shape, dtype), trainable=True),
        tfp.layers.DistributionLambda(lambda t: tfd.Independent(
            tfd.Normal(loc=t[..., :n],
                       scale= 0 + tf.nn.softplus(c + t[..., n:])),
            reinterpreted_batch_ndims=1)),
    ])
def prior_trainable(kernel_size: int, bias_size: int, dtype: any) -> tf.keras.Model:
    """Specify the prior over `keras.layers.Dense` `kernel` and `bias`."""
    n = kernel_size + bias_size
    return tf.keras.Sequential([
        tfp.layers.VariableLayer(n, dtype=dtype),  # Returns a trainable variable of shape n, regardless of input
        tfp.layers.DistributionLambda(lambda t: tfd.Independent(
            tfd.Normal(loc=t, scale=0.1),
            reinterpreted_batch_ndims=1)),
    ])

def random_gaussian_initializer(shape, dtype):
    n = int(shape / 2)
    loc_norm = tf.random_normal_initializer(mean=0., stddev=0.1)
    loc = tf.Variable(
        initial_value=loc_norm(shape=(n,), dtype=dtype)
    )
    scale_norm = tf.random_normal_initializer(mean=-3., stddev=0.1)
    scale = tf.Variable(
        initial_value=scale_norm(shape=(n,), dtype=dtype)
    )
    return tf.concat([loc, scale], 0)

def negative_loglikelihood(targets, estimated_distribution):
    return -estimated_distribution.log_prob(targets)

In [21]:
input_variational = tf.keras.layers.Input(shape=X_train.shape[1:])
hidden1_variational = tfp.layers.DenseVariational(200, activation='tanh', input_shape=(5,),
                                      make_posterior_fn=posterior_mean_field,
                                      make_prior_fn=prior_trainable,
                                      kl_weight=1. / len(y_train))(input_variational)
hidden2_variational = tfp.layers.DenseVariational(200, activation='tanh', input_shape=(5,),
                                      make_posterior_fn=posterior_mean_field,
                                      make_prior_fn=prior_trainable,
                                      kl_weight=1. / len(y_train))(hidden1_variational)
hidden3_variational = tfp.layers.DenseVariational(200, activation='tanh', input_shape=(5,),
                                      make_posterior_fn=posterior_mean_field,
                                      make_prior_fn=prior_trainable,
                                      kl_weight=1. / len(y_train))(hidden2_variational)
hidden4_variational = tfp.layers.DenseVariational(200, activation='relu', input_shape=(5,),
                                      make_posterior_fn=posterior_mean_field,
                                      make_prior_fn=prior_trainable,
                                      kl_weight=1. / len(y_train))(hidden3_variational)   

concat_variational = tf.keras.layers.Concatenate()([input_variational, hidden4_variational])

distribution_params_variational = tf.keras.layers.Dense(units=2)(concat_variational)

output_variational = tfp.layers.IndependentNormal(1)(distribution_params_variational)

model_variational = tf.keras.Model(inputs=[input_variational], outputs=[output_variational])

model_variational.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=negative_loglikelihood,metrics=[keras.metrics.RootMeanSquaredError()])

In [11]:
import tensorboard
import os

model_name = 'HSC_v6_'+str(sampling_method)+'_BNN_v1'
checkpoint_filepath = os.path.join('/models/', model_name)+'/'+model_name
log_dir = os.path.join('/logs/', model_name)
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_freq='epoch',
    save_best_only=True)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [12]:
model_variational.fit(X_train,y_train, epochs=500, shuffle = True, verbose=1, callbacks=[tensorboard_callback, model_checkpoint_callback], validation_data=(X_test, y_test))

Epoch 1/500
19686/19686 [==============================] - 124s 6ms/step - loss: nan - root_mean_squared_error: nan - val_loss: nan - val_root_mean_squared_error: nan


InvalidArgumentError: Nan in summary histogram for: dense_variational/constant_0 [Op:WriteHistogramSummary]

In [ ]:
y_predict_variational = model_variational(X_test)
y_predict_variational_mean = np.asarray(y_predict_variational.mean())[:,0]
y_predict_variational_std = np.asarray(y_predict_variational.stddev())[:,0]
# fix this

In [ ]:
#model_variational.save(checkpoint_filepath)

In [ ]:
plot_predictions(np.ravel(y_predict_variational_mean), np.ravel(y_test))

In [ ]:
sample_array_string = ""
for i in sample_array:
    sample_array_string += str(i)
    sample_array_string += "_"
    
bayesian_prediction = np.transpose(np.vstack((OID_test, y_test, y_predict_variational_mean, y_predict_variational_std)))

df = pd.DataFrame(bayesian_prediction, columns=['object_id', 'specz_redshift', 'photoz_redshift',
        'photoz_uncertainty'])
model_name = 'bnn_v1'
metrics = get_point_metrics(pd.Series(y_predict_variational_mean), pd.Series(y_test), 2.5, binned=False)

if os.path.exists(f'/predictions/HSC_v6_sampling_method_models/{model_name}') == False:
    os.makedirs(f'/predictions/HSC_v6_sampling_method_models/{model_name}')
if sampling_method == 'control':
    df.to_csv(f'/predictions/HSC_v6_sampling_method_models/{model_name}/{sampling_method}_{z_max}z_predictions.csv', index=False)
    metrics.to_csv(f'/predictions/HSC_v6_sampling_method_models/{model_name}/{sampling_method}_{z_max}z_metrics.csv', index=False)
else:
    df.to_csv(f'/predictions/HSC_v6_sampling_method_models/{model_name}/{sampling_method}_{z_max}z_predictions_{sample_array}.csv', index=False)
    metrics.to_csv(f'/predictions/HSC_v6_sampling_method_models/{model_name}/{sampling_method}_{z_max}z_metrics_{sample_array}.csv', index=False)

In [ ]:
df

In [ ]:
metrics